# 교차 검증

## 1. 개요

&nbsp;&nbsp; 교차검증이란 머신러닝의 과적합을 막기 위해 고안된 것으로, 모델을 학습시킬 때 데이터를 훈련용과 검증용으로 나눠서 교차 검증하는 방법입니다. 

## 2. 장단점

&nbsp;&nbsp; 과적합 방지가 기본적인 목적인만큼 과적합을 방지하고, 모델의 일반화 가능성을 높일 수 있다는 장점이 있습니다. 특히 데이터셋의 규모가 작을 때 더욱 유용하게 쓸 수 있습니다. 하지만 교차적으로 검증을 하니 자연스럽게 모델을 훈련하고 평가하는 비용이 증가한다는 단점이 있습니다.

## 3. 종류

&nbsp;&nbsp; **1)Hold-out Cross-Validation**

![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fbs21Ez%2FbtraImfeRz4%2Fv2XkvnLOZkKNLyMm2DRlKk%2Fimg.png)

&nbsp;&nbsp;&nbsp;&nbsp; *출처 : https://heytech.tistory.com/113*

&nbsp;&nbsp;&nbsp;&nbsp;-가장 기본적인 방법으로, 데이터셋을 훈련용과 테스트용으로 1회만 나누는 방식입니다.

&nbsp;&nbsp; **2)K-Fold Cross-Validation**

&nbsp;&nbsp;&nbsp;&nbsp; -데이터세트를 k개의 그룹(부분집합)으로 분할한 뒤 그 중 하나를 검증(validation) 데이터로, 나머지를 학습데이터로 사용해 모델을 학습시키는 과정을 k번 반복하는 방법입니다. 최종 성능 평가 결과값은 전체 k개 모델의 결과의 평균을 사용합니다.

&nbsp;&nbsp;&nbsp;&nbsp; -장단점 : 모든 데이터를 훈련과 검증에 사용하기에 과적합과 과소적합에 강하며, 모델을 일반화할 수 있는 가능성이 높습니다. 하지만 순서가 고려된 데이터가 shuffle 되어있지 않거나, 각기 다른 fold에 중복값이 있을 경우 성능이 떨어질 수 있습니다. 이러한 단점을 보완하기 위해 Stratified K-Fold와 같은 방법이 등장했습니다.

&nbsp;&nbsp;&nbsp;&nbsp; -세부 종류 : Stratified K-Fold, Group K-Fold

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; (1)Stratified K-Fold : 데이터의 클래스값(y)이 불균형한 분포도를 가진 경우, 각 폴드의 클래스를 K 개수에 맞춰 분배해 원본 데이터의 분포와 동일하게끔 데이터 세트를 분배하는 방식입니다. 일반적인 K-Fold는 클래스를 무작위로 분배하기에 그에 따른 한계점을 보완한 것입니다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; (2)Group K-Fold : 동일 그룹이 훈련 및 검증 데이터에 동시에 포함되지 않도록 하는 방법입니다.

&nbsp;&nbsp; **3)Leave-p-Out Cross-Validation(LpOCV)**

&nbsp;&nbsp; -전체 데이터의 관측값(x, y) 중에서 p개의 샘플을 선택해서 모델 검증에 활용하는 방법입니다. 훈련 및 검증 횟수는 조합 가능한 p의 갯수만큼 반복하며, K-Fold와 마찬가지로 생성된 전체 각 모델의 결과들을 평균내서 최종 결과값으로 활용합니다. 반복 횟수가 많아질 수 있는 만큼 비용이 클 수 있다는 단점이 있습니다.

&nbsp;&nbsp; **4)Leave-One-Out CV(LOOCV)**

&nbsp;&nbsp;&nbsp;&nbsp; -단 하나의 관측값(x, y)만을 검증 데이터로 활용하고 나머지는 훈련 데이터로 활용하는 방법입니다. 전체 데이터의 개수만큼 훈련 검증 횟수를 반복하기에 비용이 많이 들고 과적합 문제가 발생할 수 잇습니다.

## 4. 참고자료

&nbsp;&nbsp; https://heytech.tistory.com/113

&nbsp;&nbsp; https://scribblinganything.tistory.com/699

&nbsp;&nbsp; https://deep-learning-study.tistory.com/623

## 5. 실습

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
from sklearn import linear_model, tree, ensemble

In [2]:
train_data = pd.read_csv('C:/Users/neddy/Documents/GitHub/머신러닝학습/house_prices_train.csv')

# Remove rows with missing target values
train_data.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = train_data.SalePrice # Target variable             
train_data.drop(['SalePrice'], axis=1, inplace=True) # Removing target variable from training data

train_data.drop(['LotFrontage', 'GarageYrBlt', 'MasVnrArea'], axis=1, inplace=True) # Remove columns with null values

# Select numeric columns only
numeric_cols = [cname for cname in train_data.columns if train_data[cname].dtype in ['int64', 'float64']]
X = train_data[numeric_cols].copy()

print("Shape of input data: {} and shape of target variable: {}".format(X.shape, y.shape))

X.head() # Show first 5 training examples

Shape of input data: (1460, 34) and shape of target variable: (1460,)


,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,1,60,8450,7,5,2003,2003,706,0,150,...,548,0,61,0,0,0,0,0,2,2008
1,2,20,9600,6,8,1976,1976,978,0,284,...,460,298,0,0,0,0,0,0,5,2007
2,3,60,11250,7,5,2001,2002,486,0,434,...,608,0,42,0,0,0,0,0,9,2008
3,4,70,9550,7,5,1915,1970,216,0,540,...,642,0,35,272,0,0,0,0,2,2006
4,5,60,14260,8,5,2000,2000,655,0,490,...,836,192,84,0,0,0,0,0,12,2008


In [3]:
# Lets split the data into 5 folds.  
# We will use this 'kf'(KFold splitting stratergy) object as input to cross_val_score() method
kf =KFold(n_splits=5, shuffle=True, random_state=42)

cnt = 1
# split()  method generate indices to split data into training and test set.
for train_index, test_index in kf.split(X, y):
    print(f'Fold:{cnt}, Train set: {len(train_index)}, Test set:{len(test_index)}')
    cnt += 1

Fold:1, Train set: 1168, Test set:292
Fold:2, Train set: 1168, Test set:292
Fold:3, Train set: 1168, Test set:292
Fold:4, Train set: 1168, Test set:292
Fold:5, Train set: 1168, Test set:292


In [4]:
def rmse(score):
    rmse = np.sqrt(-score)
    print(f'rmse= {"{:.2f}".format(rmse)}')

In [5]:
#선형회귀
score = cross_val_score(linear_model.LinearRegression(), X, y, cv= kf, scoring="neg_mean_squared_error")
print(f'Scores for each fold: {score}')
rmse(score.mean())

Scores for each fold: [-1.39334669e+09 -1.32533433e+09 -3.39493937e+09 -9.31045536e+08
 -7.16620849e+08]
rmse= 39398.70


In [6]:
#의사결정나무
score = cross_val_score(tree.DecisionTreeRegressor(random_state= 42), X, y, cv=kf, scoring="neg_mean_squared_error")
print(f'Scores for each fold: {score}')
rmse(score.mean())

Scores for each fold: [-2.28396934e+09 -1.70193863e+09 -2.50505513e+09 -1.48547479e+09
 -1.66691378e+09]
rmse= 43916.63


In [7]:
#랜포
score = cross_val_score(ensemble.RandomForestRegressor(random_state= 42), X, y, cv= kf, scoring="neg_mean_squared_error")
print(f'Scores for each fold are: {score}')
rmse(score.mean())

Scores for each fold are: [-8.58316418e+08 -6.13821216e+08 -2.06121160e+09 -7.97273029e+08
 -5.68429309e+08]
rmse= 31301.92


KFold를 활용한 하이퍼파라미터 튜닝

In [8]:
max_depth = [1,2,3,4,5,6,7,8,9,10]

for val in max_depth:
    score = cross_val_score(tree.DecisionTreeRegressor(max_depth= val, random_state= 42), X, y, cv= kf, scoring="neg_mean_squared_error")
    print(f'For max depth: {val}')
    rmse(score.mean())

For max depth: 1
rmse= 58803.64
For max depth: 2
rmse= 50060.31
For max depth: 3
rmse= 42152.85
For max depth: 4
rmse= 39218.54
For max depth: 5
rmse= 40185.90
For max depth: 6
rmse= 40522.15
For max depth: 7
rmse= 41089.08
For max depth: 8
rmse= 41161.27
For max depth: 9
rmse= 41441.94
For max depth: 10
rmse= 41758.39


In [9]:
estimators = [50, 100, 150, 200, 250, 300, 350]

for count in estimators:
    score = cross_val_score(ensemble.RandomForestRegressor(n_estimators= count, random_state= 42), X, y, cv= kf, scoring="neg_mean_squared_error")
    print(f'For estimators: {count}')
    rmse(score.mean())

For estimators: 50
rmse= 31450.86
For estimators: 100
rmse= 31301.92
For estimators: 150
rmse= 31187.45
For estimators: 200
rmse= 31176.16
For estimators: 250
rmse= 31246.61
For estimators: 300
rmse= 31242.74
For estimators: 350
rmse= 31313.74
